# Variables y Constantes en Rust

En Rust, las variables se declaran principalmente con `let` y las constantes con `const`.  
La diferencia clave está en **mutabilidad**, **alcance** y **evaluación en tiempo de compilación**.

## Concepto de **ownership**

### 🔑 Concepto básico

En Rust, **ownership** (propiedad) es el sistema que garantiza seguridad de memoria sin necesidad de un garbage collector.  
Las reglas son simples:

1. Cada valor en Rust tiene un **dueño** (una variable).
2. Solo puede haber **un dueño a la vez**.
3. Cuando el dueño sale de alcance (scope), el valor se libera automáticamente.


## 🔹 `let` inmutable
- **Sintaxis**:  
  ```rust
  let x = 5;
  ```
- **Características**:
  - No puede cambiar de valor después de asignarse.
  - Vive en el *stack*.
  - Participa en el sistema de *ownership*.
  - La anotación de tipo es opcional (Rust puede inferirlo).
  - Respeta el *scope* donde fue declarado.


In [4]:
let x: i8 = 5;
x

5

In [5]:
x = 6; // error: cannot assign twice to immutable variable `x`

Error: cannot assign twice to immutable variable `x`

# 🔹 `let mut`
- **Sintaxis**:  
  ```rust
  let mut y = 10;
  y = 20; // permitido
  ```
- **Características**:
  - Puede cambiar de valor.
  - Vive en el *stack*.
  - Participa en *ownership*.
  - La anotación de tipo sigue siendo opcional.
  - Respeta el *scope*.

In [6]:
let mut y: i8 = 5;
y

5

In [7]:
y *= 2; // y = y * 2 ;
y

10

## 🔹 `const`
- **Sintaxis**:  
  ```rust
  const PI: f64 = 3.14159;
  ```
- **Características**:
  - No puede cambiar de valor.
  - Debe tener **tipo anotado** explícitamente.
  - El valor debe ser una **expresión constante** evaluada en tiempo de compilación.
  - No participa en *ownership*.
  - No respeta el *scope*: son **globales** por defecto.
  - No vive en el *stack*, sino en memoria estática.


In [3]:
const NOMBRE: &str = "Emiliano"; 
NOMBRE

"Emiliano"

## 📊 Comparación rápida

| Característica                 | `let` inmutable | `let mut` | `const` |
|--------------------------------|-----------------|-----------|---------|
| Cambia de valor                | ❌              | ✅        | ❌      |
| Vive en el stack               | ✅              | ✅        | ❌      |
| Participa en ownership         | ✅              | ✅        | ❌      |
| Necesita anotación de tipo     | Opcional        | Opcional  | ✅      |
| Valor debe ser constante       | ❌              | ❌        | ✅      |
| Respeta el scope               | ✅              | ✅        | ❌ (global) |
| Evaluado en compilación        | ❌              | ❌        | ✅      |

---
## Ownership (Propiedad)

El ownership es el sistema que usa Rust para manejar memoria **sin necesitar un garbage collector**.

### Las 3 reglas de Ownership

1. Cada valor tiene **un único dueño** (owner)
2. Solo puede haber **un dueño a la vez**
3. Cuando el dueño sale de scope, el valor es **liberado de memoria**

### 1 Scope y liberación automática

In [10]:
{
    let saludo = String::from("Hola");
    println!("{}", saludo); // válido dentro del scope
} // saludo se libera aquí automáticamente

// println!("{}", saludo); // ERROR: saludo ya no existe
println!("El String fue liberado al salir del scope");

Hola
El String fue liberado al salir del scope


### 2 Move (transferencia de ownership)

Cuando asignás un valor a otra variable, el ownership se **mueve**.
El original ya no es válido.

In [11]:
let s1 = String::from("Rust");
let s2 = s1; // el ownership se mueve a s2

// println!("{}", s1); // ERROR: s1 ya no es el dueño
println!("s2 = {}", s2); // OK

s2 = Rust


In [12]:
// Los tipos primitivos (i32, f64, bool, char) implementan Copy
// por eso sí se pueden usar después de asignar
let a = 5;
let b = a; // se copia, no se mueve
println!("a = {}, b = {}", a, b); // ambos válidos

a = 5, b = 5


### 3 Clone — copiar explícitamente

Si necesitás que ambas variables tengan el valor, usás `.clone()`.

In [13]:
let s1 = String::from("Rust");
let s2 = s1.clone(); // copia profunda

println!("s1 = {}", s1); // OK, s1 sigue siendo válido
println!("s2 = {}", s2);

s1 = Rust
s2 = Rust


## 4 Ownership en funciones

Pasar un valor a una función también **mueve** el ownership.

In [14]:
fn tomar_ownership(texto: String) {
    println!("La función recibió: {}", texto);
} // texto se libera aquí

let s = String::from("Hola desde ownership");
tomar_ownership(s); // s se mueve a la función

// println!("{}", s); // ERROR: s ya no existe

La función recibió: Hola desde ownership


In [15]:
// Una función puede devolver el ownership
fn dar_ownership() -> String {
    String::from("valor nuevo")
}

let s = dar_ownership(); // s recibe el ownership del String
println!("s = {}", s);

s = valor nuevo


## 5 Referencias y Borrowing

Para usar un valor en una función **sin transferir el ownership**, usás referencias con `&`.
Esto se llama **borrowing** (préstamo).

In [16]:
fn calcular_largo(texto: &String) -> usize {
    texto.len() // solo lee, no toma ownership
}

let s = String::from("hola mundo");
let largo = calcular_largo(&s); // prestamos s

println!("'{}' tiene {} caracteres", s, largo); // s sigue siendo válido

'hola mundo' tiene 10 caracteres


## 6 Referencias mutables

Podés prestar un valor para modificarlo con `&mut`, pero con una restricción clave:
**solo puede existir una referencia mutable a la vez**.

In [17]:
fn agregar_texto(texto: &mut String) {
    texto.push_str(", mundo!");
}

let mut s = String::from("hola");
agregar_texto(&mut s);
println!("{}", s); // "hola, mundo!"

hola, mundo!


In [18]:
// Regla: solo UNA referencia mutable a la vez
let mut s = String::from("datos");

let r1 = &mut s;
// let r2 = &mut s; // ERROR: no puede haber dos referencias mutables simultáneas

println!("r1 = {}", r1);

Error: The variable `r1` contains a reference with a non-static lifetime so
can't be persisted. You can prevent this error by making sure that the
variable goes out of scope - i.e. wrapping the code in {}.

---
## Resumen

| Concepto | Clave |
|---|---|
| `let` | Inmutable por defecto, mutable con `mut`, permite shadowing |
| `const` | Siempre inmutable, tipo obligatorio, tiempo de compilación |
| Ownership | Un solo dueño, liberación automática al salir de scope |
| Move | Asignar transfiere el ownership, el original queda inválido |
| Clone | Copia explícita y profunda del valor |
| Borrowing `&` | Préstamo inmutable, no transfiere ownership |
| Borrowing `&mut` | Préstamo mutable, solo uno a la vez |

## Let y Const

La confusión más común es pensar que let inmutable y const son lo mismo. La diferencia clave es esta:
let sin mut es inmutable, pero el valor se determina en tiempo de ejecución.
const es una constante verdadera, el valor se determina en tiempo de compilación.

### Ejemplo 1 — let puede usar valores calculados en ejecución

In [4]:
:dep rand = "0.8"

use rand::Rng;

fn ontener_numero_aleatorio() -> u8 {
    let mut rng = rand::thread_rng();
    rng.gen_range(0..=100)
}

In [6]:
// let puede recibir el resultado de una función o input del usuario
let timestamp = std::time::SystemTime::now(); // se calcula cuando el programa corre
let numero_aleatorio =  ontener_numero_aleatorio();             // depende de algo externo

// const NO puede hacer esto — esto daría ERROR:
// const TIMESTAMP: SystemTime = std::time::SystemTime::now(); // ERROR
print!("Timestamp actual: {:?}\n", timestamp);
print!("Número aleatorio: {}\n", numero_aleatorio);

Timestamp actual: SystemTime { tv_sec: 1771795773, tv_nsec: 366791066 }
Número aleatorio: 23


### Ejemplo 2 — const existe en todo el programa, let solo en su scope
rust

In [11]:
const GRAVEDAD: f64 = 9.81; // disponible en cualquier parte del código, incluso fuera de main


In [15]:

fn main() {
    let x = 5; // solo existe dentro de main
    println!("Gravedad: {}", GRAVEDAD);
    println!("x: {}", x);
}
main()

Gravedad: 9.81
x: 5


()

In [16]:

fn otra_funcion() {
    println!("Gravedad: {}", GRAVEDAD); // OK, const es global
    println!("{}", x)              // ERROR, x no existe aquí
}
otra_funcion()

Error: cannot find value `x` in this scope

### Ejemplo 3 — let permite shadowing, const no

> ⚠️ **Nota sobre este entorno (evcxr_jupyter)**
>
> El kernel de Rust para Jupyter tiene un comportamiento diferente al compilador real.
> En este entorno **podés redeclarar `const` con el mismo nombre sin error**, tanto
> en la misma celda como en celdas separadas. Esto es una decisión de diseño del kernel
> para hacer el trabajo interactivo más cómodo.
>
> En un proyecto real con `cargo` esto **no compila**:
> ```rust
> const MAX: i32 = 100;
> const MAX: i32 = 200; // ERROR en Rust real
> ```
>
> Tené esto en cuenta cuando pases código del notebook a un proyecto: lo que funciona
> aquí puede no funcionar en `cargo`.

In [18]:
let velocidad = 10;
let velocidad = velocidad * 2; // válido, es una nueva variable con el mismo nombre
println!("{}", velocidad);     // 20

20


In [ ]:
const MAX: i32 = 100;
const MAX: i32 = 200; // ERROR: no podés redeclarar una const

MAX

100

#### Resumen mental

Pensalo así: let sin mut es como una caja cerrada con llave — no podés cambiar lo que tiene adentro, pero la caja existe solo en ese momento del programa. const es como un valor grabado en piedra antes de que el programa arranque, disponible para siempre y en cualquier lugar.